In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from IPython import display
!pip install transformers
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import AutoModelForSequenceClassification
from transformers import pipeline
import torch
display.clear_output()


gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("\nRunning on device: ", str(device).upper())

if gpu_info.find('failed') >= 0 and ram_gb < 30:
  print('\nSelect the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
  print('\nTo enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  try:
      if gpu_info.find('failed') < 0:
         print(gpu_info)
  except:
    display.clear_output()
  finally:
    print('\nYour runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
    print('You are using a high-RAM runtime!')


literary_tokens = ["characterization", "character", "setting", 
                   "exposition", "climax", "resolution", "plot", 
                   "context", "action", "weapon", 
                   "danger", "death", "suspense", 
                   "emotion", "surprise", "problem", "conflict", 
                   "perspective", "transition", "relief", "metaphor", 
                   "flashback"]

subgenre_tokens = ['Vampire', 'Ghost', 'Horror', 'Comedic Horror', 'Murder', 
                   'Werewolf', 'Apocalypse','Haunted House', 'Witch', 'Hell', 
                   'Alien', 'Gore', 'Monster']


author_list = ['Brian Evenson', 'M R James', 'Ambrose Bierce', 'Elliott O Donnell', 'Joseph Sheridan Le Fanu',
               'Edgar Allan Poe', 'Bram Stoker', 'Algernon Blackwood', 
               'Madeline Gobbo', 'Nnedi Okorafor', 'Sofia Samatar', 'Franz Kafka', 
               'Laird Barron', 'Nathan Ballingrud', 'Nellie Bly', 'William Hope Hodsgon',
               'Kelly Link', 'Arthur Machen', 'George Sylvester Viereck', 'Robert Chambers', 
               'John Meade Falkner', 'Ann Radcliffe', 'Howard Lovecraft', 'Robert Louis Stevenson',
               'Edith Birkhead', 'Jeff Vandermeer', 'Henry James', 'John William Polidori', 'W Bob Holland',
               'Prest and Rymer', 'Oliver Onions', 'Stephen King', 'Clive Barker']



Running on device:  CUDA
Sat Nov 28 03:00:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                              

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("joeddav/xlm-roberta-large-xnli")
model = AutoModelForSequenceClassification.from_pretrained("joeddav/xlm-roberta-large-xnli")
generator = pipeline("zero-shot-classification", device=device, model=model, tokenizer=tokenizer)


In [ ]:
def classify_input_tokens(input_text, threshold=0.9):
    tokens=""
    #classify the lines according to literary tokens
    literary_generator = generator(input_text, literary_tokens, multi_class=True)
    for i, score in enumerate(literary_generator['scores']):
        if score > threshold:
            tokens += "["+ literary_generator['labels'][i].upper() + "]"
        else: break
    return tokens


classify_input_tokens("He cracked open to box and looked inside")

'[ACTION][PERSPECTIVE][TRANSITION][SURPRISE][EXPOSITION]'

In [ ]:
author_tokens = {'Clive Barker' : '[CLIVE BARKER]', 'J. K. Rowling' : '[J.K. ROWLING]', 'Stephen King' : '[STEPHEN KING]', 'Théophile Gautier' : '[THEOPHILE GAUTIER]', 
            'James H. Hyslop' : '[JAMES H HYSLOP]', 'Lord Edward Bulwer-Lytton' : '[LORD EDWARD BULWER-LYTTON]', 'A. T. Quiller-Couch' : '[A. T. QUILLER-COUCH]', 
            'Mrs. Margaret Oliphant' : '[MRS. MARGARET OLIPHANT]', 'Ernest Theodor Amadeus Hoffmann' : '[ERNEST THEODOR AMADEUS HOFFMAN]', 'Erckmann-Chatrian' : '[ERCKMANN-CHATRAIN]', 
            'Fiona Macleod' : '[FIONA MACLEOD]', 'Amelia B. Edwards' : '[AMELIA B. EDWARDS]', 'H. B. Marryatt' : '[H. B. MARRYATT]', 'Thomas Hardy' : '[THOMAS HARDY]', 
            'Montague Rhodes James' : '[MONTAGUE RHODES JAMES]', 'Fitz-James O\'Brien' : '[FITZ-JAMES O\'BRIEN', 'James Stephen' : '[JAMES STEPHEN]', 'Alfred Lord Tennyson' : '[ALFRED LORD TENNYSON]',
            'Amelia Edwards' : '[AMELIA EDWARDS]', 'Edward Bulwer-Lytton' : '[EDWARD BULWER-LYTTON]', 'Erckmann Chatrian' : '[ERCKMANN CHATRIAN]', 'Latifa al-Zayya' : '[LATIFA AL-ZAYYA]',
            'M. R. James' : '[M. R. JAMES]', 'Paul Brandis' : '[PAUL BRANDIS]', 'Brain Evenson' : '[BRAIN EVENSON]', 'Elliott O\'Donnell' : '[ELLIOTT O\'DONNELL]', 
            'Joseph, Sheridan Le Fanu' : '[JOSEPH, SHERIDAN LE FANU]', 'Edgar Allan Poe' : '[EDGAR ALLEN POE]', 'Bram Stoker' : '[BRAM STOKER]', 'Algernon Blackwood' :'[ALGERNON BLACKWOOD]',
            'Miles Klee' : '[MILES KLEE]', 'Nnedi Okorador' : '[NNEDI OKORADOR]', 'Sofia Samatar' : '[SOFIA SAMATAR]', 'Franz Kafka' : '[FRANZ KAFKA]', 'Laird Barron' : '[LAIRD BARRON]',
            'Nathan Ballingrud' : '[NATHAN BALLINGRUD]', 'Nellie Bly' : '[NELLIE BLY]', 'William Hop Hodgson' : '[WILLIAM HOP HODGSON]', 'Ambrose Bierce' : '[AMBROSE BIERCE]',
            'Kelly Link' : '[KELLY LINK]', 'Arthur Machen' : '[ARTHUR MACHEN]', 'George Sylvester Viereck' : '[GEORGE SYLVESTER VIERECK]', 'Robert Chambers' : '[ROBERT CHAMBERS]',
            'John Meade Falkner' : '[JOHN MEADE FALKNER]', 'Ann Radcliffe' : '[ANN RADCLIFFE]', 'Howard Lovecraft' : '[HOWARD LOVECRAFT]', 'Louis Stevenson' : '[LOUIS STEVENSON]',
            'Edith Birkhead' : '[EDITH BIRKHEAD]', 'Jeff Vandermeer' : '[JEFF VANDERMEER]', 'Henry James' : '[HENRY JAMES]', 'John William Polidori' : '[JOHN WILLIAM POLIDORI]',
            'Bob Holland' : '[BOB HOLLAND]', 'Oliver Onions' : '[OLIVER ONIONS]'}
author_inputs = f''''''
for author in author_tokens:
    author_inputs += f'''
    <div class="checkbox-container author">
        <span class="input-title">{author}</span>
        <label class="checkbox-label">
            <input type="checkbox" value="{author}">
            <span class="checkbox-custom rectangular"></span>
        </label>
    </div>
    '''
# print(author_inputs)
subgenre_tokens = ['Vampire', 'Ghost', 'Horror', 'Comedic Horror', 'Murder', 
                   'Werewolf', 'Apocalypse','Haunted House', 'Witch', 'Hell', 
                   'Alien', 'Gore', 'Monster']
genre_inputs = f''''''
for genre in subgenre_tokens:
    genre_inputs += f'''
    <div class="checkbox-container genre">
        <span class="input-title">{genre}</span>
        <label class="checkbox-label">
            <input type="checkbox" value="{genre}">
            <span class="checkbox-custom rectangular"></span>
        </label>
    </div>
    '''
# print(genre_inputs)
blacklist_path = "/content/drive/MyDrive/CSCI 470 Project/Epic/Data/Cleaned_Black_List.txt"
blacklist_words = f''''''
with open(blacklist_path, "r") as reader:
    lines = reader.readlines()
    for line in lines:
        blacklist_words += f'''<li class="blacklist" data-word="{line.lstrip().rstrip()}"><i class="fas fa-minus-circle delete"></i></li>\n'''

print(blacklist_words)

<li class="blacklist" data-word="4r5e"><i class="fas fa-minus-circle delete"></i></li>
<li class="blacklist" data-word="50 yard cunt punt"><i class="fas fa-minus-circle delete"></i></li>
<li class="blacklist" data-word="5h1t"><i class="fas fa-minus-circle delete"></i></li>
<li class="blacklist" data-word="5hit"><i class="fas fa-minus-circle delete"></i></li>
<li class="blacklist" data-word="a_s_s"><i class="fas fa-minus-circle delete"></i></li>
<li class="blacklist" data-word="a2m"><i class="fas fa-minus-circle delete"></i></li>
<li class="blacklist" data-word="a$$"><i class="fas fa-minus-circle delete"></i></li>
<li class="blacklist" data-word="a55"><i class="fas fa-minus-circle delete"></i></li>
<li class="blacklist" data-word="a$$hole"><i class="fas fa-minus-circle delete"></i></li>
<li class="blacklist" data-word="a55hole"><i class="fas fa-minus-circle delete"></i></li>
<li class="blacklist" data-word="adult"><i class="fas fa-minus-circle delete"></i></li>
<li class="blacklist" dat